# Arm Visualizer
Uses Plotly to visualize the configuration of an arm given a set of joint angles.

### Adjust Path
In order to use the RobotArm class located in `inverse_kinematics_prediction/utils/robot_arm.py` we need to add this project to our system path.

In [44]:
import sys
import os

# Add the parent directory (project root) relative to the current working directory.
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working directory:", os.getcwd())
print("Project root added to sys.path:", project_root)

Current working directory: c:\Users\cjsta\git\inverse_kinematics_prediction\notebooks
Project root added to sys.path: c:\Users\cjsta\git\inverse_kinematics_prediction


### Create RobotArm Object

In [45]:
import numpy as np

from utils.robot_arm import RobotArm

# Create a 4-DOF robot arm with 5 links, each of length 1.0 unit.
num_joints = 3
joint_limits = [(0, 2*np.pi)] * num_joints
rotation_axes = ['z', 'y', 'y']
link_lengths = [1.0] * (num_joints + 1)
link_axes = ['z', 'x', 'x', 'x']

# Instantiate the RobotArm object
robot_arm = RobotArm(num_joints=num_joints, 
                     joint_limits=joint_limits, 
                     link_lengths=link_lengths, 
                     rotation_axes=rotation_axes,
                     link_axes=link_axes)

# Test forward kinematics
test_angles = robot_arm.sample_random_joint_angles()
pos, quat = robot_arm.forward_kinematics(test_angles)
print("Joint angles (radians): ", test_angles)
print("End Effector Position (point x,y,z): ", pos)
print("End Effector Orientation (quaternion x,y,z,w): ", quat)

Joint angles (radians):  [5.94161889 2.96654724 2.11754579]
End Effector Position (point x,y,z):  [-0.243389    0.08652486  2.68926671]
End Effector Orientation (quaternion x,y,z,w):  (np.float64(-0.09589969557208287), np.float64(-0.5560588354591234), np.float64(-0.1403128063295491), np.float64(0.8135810569801584))


In [ ]:
# Forward Kinematics via RobotArm
forward_kinematics = lambda q: robot_arm.forward_kinematics(q)[0]

def generate_data(num_samples=1000):
    data_list = []
    for _ in range(num_samples):
        q = robot_arm.sample_random_joint_angles()
        xy = forward_kinematics(q)[:2]  # Take only x, y
        edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long).t()
        x = torch.tensor(np.array([xy, xy, xy]), dtype=torch.float)
        y = torch.tensor(np.tile(q, (3, 1)), dtype=torch.float)
        data_list.append(Data(x=x, edge_index=edge_index, y=y))
    return data_list

dataset = generate_data(1000)

class IK_GNN(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=16, output_dim=3):
        super(IK_GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = IK_GNN()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

def train_gnn(model, dataset, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in dataset:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {total_loss / len(dataset):.6f}")

train_gnn(model, dataset)

model.eval()
test_q = robot_arm.sample_random_joint_angles()
test_xy = forward_kinematics(test_q)[:2]
test_x = torch.tensor(np.array([test_xy, test_xy, test_xy]), dtype=torch.float)
test_edge_index = torch.tensor([[0, 1], [1, 2]], dtype=torch.long).t()
test_y = torch.tensor(np.tile(test_q, (3, 1)), dtype=torch.float)
test_data = Data(x=test_x, edge_index=test_edge_index, y=test_y)
predicted_q = model(test_data.x, test_data.edge_index)
final_pred_q = predicted_q[0].detach().numpy()
print("Predicted Joint Angles (from node 0):", final_pred_q)
print("Actual Joint Angles:", test_q)

### Visualize Sample
Let's visualize a sample in 3D.

In [46]:
import numpy as np
import plotly.graph_objects as go

# --- Helper Functions for Plotly Visualization ---

def get_rotation_matrix_for_axis(axis_str):
    """
    Return a 3x3 rotation matrix that rotates a vector from the default (z-axis)
    to be aligned with the specified axis ('x', 'y', or 'z').
    Here the default cylinder is built along the z-axis.
    """
    axis_str = axis_str.lower()
    if axis_str == 'x':
        # Rotate from z to x: rotate -90° about y.
        angle = -np.pi/2
        R = np.array([
            [ np.cos(angle), 0, np.sin(angle)],
            [ 0,             1, 0],
            [-np.sin(angle), 0, np.cos(angle)]
        ])
    elif axis_str == 'y':
        # Rotate from z to y: rotate +90° about x.
        angle = np.pi/2
        R = np.array([
            [1, 0,             0],
            [0, np.cos(angle), -np.sin(angle)],
            [0, np.sin(angle),  np.cos(angle)]
        ])
    elif axis_str == 'z':
        R = np.eye(3)
    else:
        raise ValueError("Invalid axis string. Must be 'x', 'y', or 'z'.")
    return R

def cylinder_mesh(center, axis_str, radius=0.05, height=0.15, resolution=20):
    """
    Compute mesh grids (X, Y, Z) for a cylinder built along the default z-axis and then
    rotated so its long axis aligns with the specified axis (axis_str). The mesh is then translated 
    to the provided center.
    """
    z = np.linspace(-height/2, height/2, resolution)
    theta = np.linspace(0, 2*np.pi, resolution)
    theta_grid, z_grid = np.meshgrid(theta, z)
    x_grid = radius * np.cos(theta_grid)
    y_grid = radius * np.sin(theta_grid)
    # Flatten the grid for transformation.
    points = np.vstack((x_grid.ravel(), y_grid.ravel(), z_grid.ravel()))
    
    # Rotate from default z-axis to desired axis.
    R_align = get_rotation_matrix_for_axis(axis_str)
    rotated_points = R_align.dot(points)
    
    # Reshape and translate.
    X = rotated_points[0, :].reshape(theta_grid.shape) + center[0]
    Y = rotated_points[1, :].reshape(theta_grid.shape) + center[1]
    Z = rotated_points[2, :].reshape(theta_grid.shape) + center[2]
    
    return X, Y, Z

def quaternion_to_rotation_matrix(quat):
    """
    Convert a quaternion (in ROS format: (x, y, z, w)) to a 3x3 rotation matrix.
    """
    x, y, z, w = quat
    R = np.array([
        [1 - 2*(y**2 + z**2),   2*(x*y - z*w),       2*(x*z + y*w)],
        [2*(x*y + z*w),         1 - 2*(x**2 + z**2), 2*(y*z - x*w)],
        [2*(x*z - y*w),         2*(y*z + x*w),       1 - 2*(x**2 + y**2)]
    ])
    return R

# --- Visualization Code using Plotly ---
# Assume:
# - robot_arm is an instance of RobotArm (already created).
# - joint_angles is a given array (or list) of joint angles.
# For example:
# joint_angles = robot_arm.sample_random_joint_angles()

# Get the full chain of poses.
poses = robot_arm.get_joint_poses(test_angles)
# Extract positions (each pose is a tuple (position, quaternion)).
positions = np.array([pose[0] for pose in poses])  # shape: (num_joints+2, 3)

# Compute the overall data bounds.
x_min, x_max = positions[:,0].min(), positions[:,0].max()
y_min, y_max = positions[:,1].min(), positions[:,1].max()
z_min, z_max = positions[:,2].min(), positions[:,2].max()
x_center = (x_min + x_max) / 2
y_center = (y_min + y_max) / 2
z_center = (z_min + z_max) / 2
max_range = max(x_max - x_min, y_max - y_min, z_max - z_min)
half_range = max_range / 2

# Set the axis range as a cube centered on the data.
x_range = [x_center - half_range, x_center + half_range]
y_range = [y_center - half_range, y_center + half_range]
z_range = [z_center - half_range, z_center + half_range]

# Create a Plotly figure.
fig = go.Figure()

# Plot the links as a 3D line.
fig.add_trace(go.Scatter3d(
    x = positions[:,0],
    y = positions[:,1],
    z = positions[:,2],
    mode = 'lines+markers',
    line = dict(color='black', width=4),
    marker = dict(size=2),
    name = 'Links'
))

# Plot each joint as a solid cylinder.
joint_radius = 0.05
joint_height = 0.15
for i in range(1, len(poses)-1):  # Skip base and optionally the end-effector.
    center = poses[i][0]
    axis_str = robot_arm.rotation_axes[i-1]  # Each joint's rotation axis.
    X, Y, Z = cylinder_mesh(center, axis_str, radius=joint_radius, height=joint_height, resolution=20)
    # Use a constant surface color for a solid appearance.
    surface_color = np.ones_like(X)
    fig.add_trace(go.Surface(
        x = X, y = Y, z = Z,
        surfacecolor = surface_color,
        colorscale = [[0, 'gray'], [1, 'gray']],
        cmin = 0, cmax = 1,
        showscale = False,
        opacity = 1,
        name = f'Joint {i}'
    ))

# Plot the end-effector pose as a coordinate frame.
end_effector_pos, end_effector_quat = poses[-1]
R_eff = robot_arm._quaternion_to_rotation_matrix(end_effector_quat)
frame_axis_length = 0.2
x_axis_vec = R_eff[:,0] * frame_axis_length
y_axis_vec = R_eff[:,1] * frame_axis_length
z_axis_vec = R_eff[:,2] * frame_axis_length

fig.add_trace(go.Scatter3d(
    x = [end_effector_pos[0], end_effector_pos[0] + x_axis_vec[0]],
    y = [end_effector_pos[1], end_effector_pos[1] + x_axis_vec[1]],
    z = [end_effector_pos[2], end_effector_pos[2] + x_axis_vec[2]],
    mode = 'lines',
    line = dict(color='red', width=5),
    name = 'End-effector X'
))
fig.add_trace(go.Scatter3d(
    x = [end_effector_pos[0], end_effector_pos[0] + y_axis_vec[0]],
    y = [end_effector_pos[1], end_effector_pos[1] + y_axis_vec[1]],
    z = [end_effector_pos[2], end_effector_pos[2] + y_axis_vec[2]],
    mode = 'lines',
    line = dict(color='green', width=5),
    name = 'End-effector Y'
))
fig.add_trace(go.Scatter3d(
    x = [end_effector_pos[0], end_effector_pos[0] + z_axis_vec[0]],
    y = [end_effector_pos[1], end_effector_pos[1] + z_axis_vec[1]],
    z = [end_effector_pos[2], end_effector_pos[2] + z_axis_vec[2]],
    mode = 'lines',
    line = dict(color='blue', width=5),
    name = 'End-effector Z'
))

# Update the layout with a square figure and set the axis ranges to form a cube.
fig.update_layout(
    width=800,
    height=800,
    scene=dict(
        xaxis=dict(title="X", range=x_range),
        yaxis=dict(title="Y", range=y_range),
        zaxis=dict(title="Z", range=z_range),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)
    ),
    title="3D Visualization of Manipulator Configuration (Plotly)"
)

fig.show()
